In [1]:
import time

import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from os.path import join
from os.path import dirname
from src.components.eval import eval_env
from src.components.train import train_env
from src.components.tune import tune_hyper_param
from src.envs.creator import env_creator
from src.envs.creator import register_platform_env
from src.files.utills import pickle_obj

/home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


# Define Configs

In [2]:
total_workers = 12
num_envs_per_worker = 24
algo = "PPO"
env_name = "Platform-v0"
register_platform_env(env_name)
config = (
    PPOConfig()
    .environment(env_name)
    .training(gamma=0.995, num_sgd_iter=10, sgd_minibatch_size=1000, clip_param=0.1, lr=1e-4, train_batch_size=2000,
              entropy_coeff=1e-4)
    .resources(num_gpus=1, num_cpus_per_worker=1)
    .rollouts(num_rollout_workers=total_workers, num_envs_per_worker=num_envs_per_worker)
    .framework("torch")
    .training(
        model={"fcnet_hiddens": [64, 64, 64], "vf_share_layers": False, "use_lstm": tune.grid_search([True, False]),
               "lstm_cell_size": 32,
               "max_seq_len": 5})
    .evaluation(evaluation_num_workers=1)
)
log_dir = "tmp/pipeline_logs"

# Tune Hyper-parameters

The hyper-parameters to be optimized can be defined in the config using tune API. For example in this case *use_lstm* is a hyper-parameter with values *[True, False]*. The *tune_hyper_param*  searches for the optimal parameter values and returns the best config.

In [3]:
tune_results = tune_hyper_param(
    algo=algo,
    config=config,
    log_dir=log_dir,
    iterations=30,
    name="platform_solver_tuning"
)

best_config = tune_results.get_best_result(metric="episode_reward_mean", mode="max").config

2023-02-19 11:35:23,401	INFO worker.py:1538 -- Started a local Ray instance.


2023-02-19 11:35:24,611	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-02-19 11:35:24,612	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
(pid=45334) /home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=45334)   if (distutils.version.LooseVersion(tf.__version__) <
(PPO pid=45334) 2023-02-19 11:35:28,147	

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_Platform-v0_1e4a6_00000,3984,"{'num_env_steps_sampled': 3984, 'num_env_steps_trained': 3984, 'num_agent_steps_sampled': 3984, 'num_agent_steps_trained': 3984}",{},2023-02-19_11-35-48,False,2.15099,{},0.785356,0.136186,0,1722,1722,0cb2d8f9e64a4bc1b2dc1073b6e68e92,hamza-Legion-5-15ACH6H,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000007, 'cur_lr': 9.999999999999998e-05, 'total_loss': 0.06541708384950956, 'policy_loss': -0.010283544358874982, 'vf_loss': 0.07576281701525052, 'vf_explained_var': -0.046028035879135135, 'kl': 0.002434352423791945, 'entropy': 5.49059796333313, 'entropy_coeff': 9.999999999999998e-05}, 'model': {}, 'num_grad_updates_lifetime': 15.5, 'diff_num_grad_updates_vs_sampler_policy': 14.5}}, 'num_env_steps_sampled': 3984, 'num_env_steps_trained': 3984, 'num_agent_steps_sampled': 3984, 'num_agent_steps_trained': 3984}",1,192.168.0.185,3984,3984,3984,3984,3984,3984,0,12,0,0,3984,"{'cpu_util_percent': 69.23636363636363, 'ram_util_percent': 81.91818181818182}",45334,{},{},{},"{'mean_raw_obs_processing_ms': 24.829659580832928, 'mean_inference_ms': 3.315555245956048, 'mean_action_processing_ms': 5.028468996145276, 'mean_env_wait_ms': 355.56005124072476, 'mean_env_render_ms': 0.0}",7.29708,7.29708,7.29708,"{'training_iteration_time_ms': 7280.924, 'load_time_ms': 35.576, 'load_throughput': 111986.028, 'learn_time_ms': 866.21, 'learn_throughput': 4599.345, 'synch_weights_time_ms': 7.528}",1676802948,0,3984,1,1e4a6_00000,13.2726


2023-02-19 11:40:23,185	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-02-19 11:40:23,186	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
(pid=46779) /home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=46779)   if (distutils.version.LooseVersion(tf.__version__) <
(PPO pid=46779) 2023-02-19 11:40:27,295	

# Train RL Agent

The model is trained here using the best config from the tune step. The best training checkpoint is then chosen for evaluation

In [4]:
train_results = train_env(
    algo=algo,
    config=best_config,
    log_dir=log_dir,
    iterations=150,
    stop_reward_mean=0.99,
    name="platform_solver"
)
best_checkpoint = train_results.get_best_result(metric="episode_reward_mean", mode="max").best_checkpoints[0]
best_checkpoint_path = best_checkpoint[0]._local_path

2023-02-19 11:45:04,031	WARNING algorithm_config.py:488 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-02-19 11:45:04,032	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-02-19 11:45:04,032	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
(pid=48237) /home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are d

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_Platform-v0_77a94_00000,3984,"{'num_env_steps_sampled': 3984, 'num_env_steps_trained': 3984, 'num_agent_steps_sampled': 3984, 'num_agent_steps_trained': 3984}",{},2023-02-19_11-45-29,False,2.11469,{},0.596098,0.140212,0,1770,1770,5d2b7732ea78453785e8d67dfba8e686,hamza-Legion-5-15ACH6H,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000007, 'cur_lr': 9.999999999999998e-05, 'total_loss': -0.0034081825617856038, 'policy_loss': -0.017032664505920062, 'vf_loss': 0.013551545639832815, 'vf_explained_var': 0.08442758123079935, 'kl': 0.00300189311481193, 'entropy': 5.274435647328695, 'entropy_coeff': 9.999999999999998e-05}, 'model': {}, 'num_grad_updates_lifetime': 15.5, 'diff_num_grad_updates_vs_sampler_policy': 14.5}}, 'num_env_steps_sampled': 3984, 'num_env_steps_trained': 3984, 'num_agent_steps_sampled': 3984, 'num_agent_steps_trained': 3984}",1,192.168.0.185,3984,3984,3984,3984,3984,3984,0,12,0,0,3984,"{'cpu_util_percent': 69.78333333333333, 'ram_util_percent': 83.35}",48237,{},{},{},"{'mean_raw_obs_processing_ms': 27.78589189113024, 'mean_inference_ms': 3.501423698706021, 'mean_action_processing_ms': 5.423940185116941, 'mean_env_wait_ms': 408.8159345596161, 'mean_env_render_ms': 0.0}",8.32082,8.32082,8.32082,"{'training_iteration_time_ms': 8302.204, 'load_time_ms': 35.53, 'load_throughput': 112131.061, 'learn_time_ms': 1067.695, 'learn_throughput': 3731.401, 'synch_weights_time_ms': 9.418}",1676803529,0,3984,1,77a94_00000,13.0848


2023-02-19 12:02:12,037	INFO tune.py:762 -- Total run time: 1028.01 seconds (1027.37 seconds for the tuning loop).


# Evaluate Trained Agent

The best checkpoint is evaluated for and the results are returned as dataframe for further interpretations

In [5]:
print(best_checkpoint_path)
start = time.time()
results = eval_env(ray=ray, epochs=1000, workers=8, env_creator=env_creator, env_kwargs=dict(env_config=''),
                   checkpoint_path=best_checkpoint_path)
print(time.time() - start)

/home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


/home/hamza/PycharmProjects/platform-solver/tmp/pipeline_logs/platform_solver/PPO_Platform-v0_77a94_00000_0_2023-02-19_11-45-04/checkpoint_000110


(EvaluationActor pid=50706) /home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(EvaluationActor pid=50706)   if (distutils.version.LooseVersion(tf.__version__) <
(EvaluationActor pid=50703) /home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(EvaluationActor pid=50703)   if (distutils.version.LooseVersion(tf.__version__) <
(EvaluationActor pid=50707) /home/hamza/PycharmProjects/platform-solver/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(EvaluationActor pid=50707)   if (distutils.version.LooseVersion(tf.__version__) <
(EvaluationActor

24.52687692642212


In [6]:
pickle_obj(results,join('tmp','evaluation_results.pkl'))